In [2]:
import onnx
import onnxruntime as ort
import numpy as np
import onnx_graphsurgeon as gs
import os

In [13]:
CNN= "./models/CNN.onnx"

In [14]:
model = onnx.load(CNN)
graph = gs.import_onnx(model)


In [15]:
graph.inputs, graph.outputs


([Variable (CNN_input): (shape=[1, 3, 64, 64], dtype=float32)],
 [Variable (Predicted_leaf_disease): (shape=[1, 10], dtype=float32)])

In [16]:
inter_nodes=[nodes.name for nodes in graph.nodes]
inter_nodes

['/conv_block_1/conv_block_1.0/Conv',
 '/conv_block_1/conv_block_1.1/Relu',
 '/conv_block_1/conv_block_1.2/Conv',
 '/conv_block_1/conv_block_1.3/Relu',
 '/conv_block_1/conv_block_1.4/MaxPool',
 '/conv_block_2/conv_block_2.0/Conv',
 '/conv_block_2/conv_block_2.1/Relu',
 '/conv_block_2/conv_block_2.2/Conv',
 '/conv_block_2/conv_block_2.3/Relu',
 '/conv_block_2/conv_block_2.4/MaxPool',
 '/classifier/classifier.0/Flatten',
 '/classifier/classifier.1/Gemm']

In [17]:
def add_intermediate_outputs(model_path):
  

    model = onnx.load(model_path)
    graph = gs.import_onnx(model)

    
    # Add intermediate outputs
    for node in graph.nodes:
        if node.name in inter_nodes:
           
            output_name = f"{node.name}_output"
            
            original_output = node.outputs[0]
            dtype = original_output.dtype
            shape = original_output.shape
            
            
            new_tensor = gs.Variable(
                name=output_name,
                dtype=dtype if dtype is not None else np.float32,
                shape=shape
            )
            
            graph.outputs.append(new_tensor)
            
            
            identity_node = gs.Node(
                op="Identity",
                name=f"{node.name}_identity",
                inputs=[node.outputs[0]],
                outputs=[new_tensor]
            )
            
            graph.nodes.append(identity_node)
            
    
  
    modified_model_path = model_path.replace('.onnx', '_with_outputs.onnx')
    onnx.save(gs.export_onnx(graph), modified_model_path)
    print(f"Modified model saved with  intermediate outputs")
    return modified_model_path


def save_layer_outputs(session, input_data, output_dir="layer_outputs"):
    
    """
    Run inference and save all layer outputs as .npy files
    
    Args:
        session: ONNX Runtime session
        input_data: Input numpy array
        output_dir: Directory to save layer outputs
    """
   
    os.makedirs(output_dir, exist_ok=True)
    
    
    input_name = session.get_inputs()[0].name
    output_names = [output.name for output in session.get_outputs()]
    
    outputs = session.run(output_names, {input_name: input_data})
    

    for layer_name, output in zip(output_names, outputs):
        layer_name = layer_name.replace('/', '_')
        output_path = os.path.join(output_dir, f"{layer_name}.raw")
        output.tofile(output_path)
        print(f"Saved {layer_name} with shape {output.shape} to {output_path}")


In [ ]:

CNN = "./models/CNN.onnx"    

modified_model_path = add_intermediate_outputs(CNN)

session = ort.InferenceSession(modified_model_path)

input_data = np.random.randn(1, 3, 64, 64).astype(np.float32)

save_layer_outputs(session, input_data)



Modified model saved with  intermediate outputs
Saved Predicted_leaf_disease with shape (1, 10) to layer_outputs\Predicted_leaf_disease.raw
Saved _conv_block_1_conv_block_1.0_Conv_output with shape (1, 10, 64, 64) to layer_outputs\_conv_block_1_conv_block_1.0_Conv_output.raw
Saved _conv_block_1_conv_block_1.1_Relu_output with shape (1, 10, 64, 64) to layer_outputs\_conv_block_1_conv_block_1.1_Relu_output.raw
Saved _conv_block_1_conv_block_1.2_Conv_output with shape (1, 10, 64, 64) to layer_outputs\_conv_block_1_conv_block_1.2_Conv_output.raw
Saved _conv_block_1_conv_block_1.3_Relu_output with shape (1, 10, 64, 64) to layer_outputs\_conv_block_1_conv_block_1.3_Relu_output.raw
Saved _conv_block_1_conv_block_1.4_MaxPool_output with shape (1, 10, 32, 32) to layer_outputs\_conv_block_1_conv_block_1.4_MaxPool_output.raw
Saved _conv_block_2_conv_block_2.0_Conv_output with shape (1, 10, 32, 32) to layer_outputs\_conv_block_2_conv_block_2.0_Conv_output.raw
Saved _conv_block_2_conv_block_2.1_Rel

In [19]:
for output in session.get_outputs():
    print(output.name)


Predicted_leaf_disease
/conv_block_1/conv_block_1.0/Conv_output
/conv_block_1/conv_block_1.1/Relu_output
/conv_block_1/conv_block_1.2/Conv_output
/conv_block_1/conv_block_1.3/Relu_output
/conv_block_1/conv_block_1.4/MaxPool_output
/conv_block_2/conv_block_2.0/Conv_output
/conv_block_2/conv_block_2.1/Relu_output
/conv_block_2/conv_block_2.2/Conv_output
/conv_block_2/conv_block_2.3/Relu_output
/conv_block_2/conv_block_2.4/MaxPool_output
/classifier/classifier.0/Flatten_output
/classifier/classifier.1/Gemm_output


In [3]:
np.fromfile('./layer_outputs/_classifier_classifier.0_Flatten_output.raw')

array([1.16512433e-001, 5.13032150e-315, 0.00000000e+000, ...,
       3.21901758e+004, 5.06043204e+004, 5.22737208e+003], shape=(1280,))